#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet

import pandas as pd
import numpy as np

In [ ]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [ ]:
final_dataset.head()

In [ ]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

In [ ]:
duplicates = final_dataset.duplicated(subset=['product_id', 'periodo'])
duplicate_rows = final_dataset[duplicates]

if duplicate_rows.empty:
    print("No hay registros duplicados por product_id y periodo.")
else:
    print("Registros duplicados encontrados por product_id y periodo:")
    display(duplicate_rows)


#### Funcion para crear los modelos

#### Armado de los modelos

In [ ]:
final_dataset['periodo'] = final_dataset['periodo'].astype(str)
final_dataset['periodo'] = pd.to_datetime(final_dataset['periodo'], format='%Y%m', errors='coerce')

In [29]:
import os
import joblib
from neuralprophet import NeuralProphet

product_ids = final_dataset['product_id'].unique()
predictions = []
    
for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[['periodo', 'y']]
    
    product_data = product_data.rename(columns={'periodo': 'ds', 'y': 'y'})
    # product_data['y'] = np.log1p(product_data['y'])
    # product_data['y'] = np.log1p(product_data['y'])
    
    # try:
    model = NeuralProphet(
        # yearly_seasonality=True,
        # n_changepoints=20,
        learning_rate=0.001,
        epochs=200,
        n_forecasts= 2,
        loss_func='MSE'
    )
    model.fit(product_data)
    # except:
    #     display(product_data)
    
    future = model.make_future_dataframe(product_data, periods=2)
    forecast = model.predict(future)
    
    # forecast['yhat1'] = np.expm1(forecast['yhat1'])
    forecast['yhat1'] = forecast['yhat1'].clip(lower=0)

    os.makedirs('NeuralProphet_results', exist_ok=True)
    joblib.dump(model, f'NeuralProphet_results/model_product_{product_id}.pkl')
    print(forecast.head())  # Imprime las primeras filas del DataFrame forecast

    predicted_y = forecast.iloc[-1]['yhat1']
    
    if predicted_y == 0:
        predicted_y = np.mean(product_data['y'])
    
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}. Numero de predicciones ${len(predictions)}')

Epoch 200: 100%|██████████| 200/200 [00:00<00:00, 14163.25it/s, loss=0.675, v_num=3114, MAE=48.90, RMSE=54.70, Loss=0.675, RegLoss=0.000] 


INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [80.]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,          

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 210.26it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


          ds     y       yhat1       trend
0 2020-01-01  None  167.152527  167.152527
1 2020-02-01  None  184.353027  184.353027
Modelo para el producto 20213 entrenado y guardado. Predicción a 2 meses: 184.35302734375. Numero de predicciones $757


...

Epoch 200: 100%|██████████| 200/200 [00:00<00:00, 12156.70it/s, loss=0.191, v_num=3137, MAE=0.230, RMSE=0.255, Loss=0.191, RegLoss=0.000] 


INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [75.]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,          

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 174.04it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


          ds     y     yhat1     trend
0 2020-01-01  None  0.623023  0.623023
1 2020-02-01  None  0.698025  0.698025
Modelo para el producto 21214 entrenado y guardado. Predicción a 2 meses: 0.6980247497558594. Numero de predicciones $780


In [30]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
